In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from snmfem.conf import DATASETS_PATH

from snmfem.models import EDXS

In [ ]:
# Load an EDXS model
model_parameters = {"params_dict" : {"c0" : 4.8935e-05, 
                                          "c1" : 1464.19810,
                                          "c2" : 0.04216872,
                                          "b0" : 0.15910789,
                                          "b1" : -0.00773158,
                                          "b2" : 8.7417e-04},
                         "db_name" : "simple_xrays_threshold.json",
                         "e_offset" : 0.208,
                         "e_scale" : 0.01,
                         "e_size": 1980,
                         "width_slope" : 0.01,
                         "width_intercept" : 0.065,
                         "seed" : 1}
model = EDXS(**model_parameters)
e = model.x

You need to run the script `python script/generate_synthetic_dataset.py` in order to get the data for the next cell.

In [ ]:
# load an EDXS sample
sample = DATASETS_PATH / Path("aspim037_N100_2ptcls_brstlg") / Path("sample_0.npz")
# load data
data = np.load(sample)
X = data["X"]
Xdot = data["Xdot"]
nx, ny, ns = X.shape
Xflat = X.transpose([2,0,1]).reshape(ns, nx*ny)
Xdotflat = Xdot.transpose([2,0,1]).reshape(ns, nx*ny)
densities = data["densities"]
phases = data["phases"]
true_spectra = np.expand_dims(densities, axis=1) * phases
true_maps = data["weights"]
k = true_maps.shape[2]
true_maps_flat = true_maps.transpose([2,0,1]).reshape(k,nx*ny)
assert(true_maps.shape[:2] == (nx, ny))
G = data["G"]


In [ ]:
A = true_maps.reshape(-1, true_maps.shape[2]).T
GP = np.linalg.lstsq(A.T, Xflat.T)[0]
P = np.linalg.lstsq(G, GP.T)[0]


In [ ]:
# parameters
c0 = 4.8935e-05
c1 = 1464.19810
c2 = 0.04216872
b0 = 0.15910789
b1 = -0.00773158
b2 = 8.7417e-04

# Numpy implementation


In [ ]:
def Bmatrix(b0, b1, b2, c0, c1, c2, e):
    beta = b0/e + b1 + b2*e
    Gamma = np.exp(-c2 / e**3) * (1 - np.exp(-c1 / e**3))
    alpha = e**3 * (1 - np.exp(- c0 / e**3)) / c0
    B = np.expand_dims(beta * Gamma * alpha, axis=1 )
    return B

def loss(b0, b1, b2, c0, c1, c2, e, G, P, A, X):
    B = Bmatrix(b0, b1, b2, c0, c1, c2, e)
    GP = G @ P
    GPA = (GP + B) @ A
    return - np.sum(X*np.log(GPA)) + np.sum(GPA)

In [ ]:
B = Bmatrix(b0, b1, b2, c0, c1, c2, e)
l = loss(b0, b1, b2, c0, c1, c2, e, G, P, A, Xflat)


# Torch implementation

In [ ]:
import torch
from torch.autograd import Variable


In [ ]:
t_c0 = Variable(torch.tensor(c0), requires_grad=True)
t_c1 = Variable(torch.tensor(c1), requires_grad=True)
t_c2 = Variable(torch.tensor(c2), requires_grad=True)
t_b0 = Variable(torch.tensor(b0), requires_grad=True)
t_b1 = Variable(torch.tensor(b1), requires_grad=True)
t_b2 = Variable(torch.tensor(b2), requires_grad=True)

t_e = Variable(torch.tensor(e), requires_grad=False)

t_G = Variable(torch.tensor(G), requires_grad=False)
t_P = Variable(torch.tensor(P), requires_grad=False)
t_A = Variable(torch.tensor(A), requires_grad=False)
t_X = Variable(torch.tensor(Xflat), requires_grad=False)


In [ ]:
def torch_Bmatrix(b0, b1, b2, c0, c1, c2, e):
    beta = b0/e + b1 + b2*e
    Gamma = torch.exp(-c2 / e**3) * (1 - torch.exp(-c1 / e**3))
    alpha = e**3 * (1 - torch.exp(- c0 / e**3)) / c0
    B = torch.unsqueeze(beta * Gamma * alpha, axis=1 )
    return B

def torch_loss(b0, b1, b2, c0, c1, c2, e, G, P, A, X):
    B = torch_Bmatrix(b0, b1, b2, c0, c1, c2, e)
    GP = torch.matmul(G, P)
    GPA = torch.matmul((GP + B), A )
    return - torch.sum(X*torch.log(GPA)) + torch.sum(GPA)

### Check that the forward functions gives the same results in torch and in numpy

In [ ]:
B_torch = torch_Bmatrix(t_b0, t_b1, t_b2, t_c0, t_c1, t_c2, t_e)

np.linalg.norm(B - B_torch.detach().numpy())/np.linalg.norm(B)

In [ ]:
l_torch = torch_loss(t_b0, t_b1, t_b2, t_c0, t_c1, t_c2, t_e, t_G, t_P, t_A, t_X)

np.linalg.norm(l - l_torch.detach().numpy())/np.linalg.norm(l)

### Compute the gradients

In [ ]:
l_torch = torch_loss(t_b0, t_b1, t_b2, t_c0, t_c1, t_c2, t_e, t_G, t_P, t_A, t_X)
l_torch.backward()

# here they are! The number are quite big indeed. That is likely to cause problem for the optimization...
t_c0.grad, t_c1.grad, t_c2.grad, t_b0.grad, t_b1.grad,t_b2.grad